In [1]:
import sys
import os

# Add project root and parent directory to path
project_root = os.path.abspath("..")
parent_dir = os.path.abspath(os.path.join("..", ".."))

if project_root not in sys.path:
    sys.path.append(project_root)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from sbmlParser.parser import ParseSBMLFile
from sbml_rust_generator import SbmlToRustConverter

In [2]:
sbml_file = "../data/PBPK_BPA_model.sbml"

# Parse the SBML file into a dictionary
model_data = ParseSBMLFile(sbml_file)

print(f"Loaded model with {len(model_data['species'])} species and {len(model_data['reactions'])} reactions.")

Loaded model with 1 species and 1 reactions.


In [3]:
converter = SbmlToRustConverter(model_data)

# Check model info
info = converter.get_model_info()
for key, value in info.items():
    print(f"{key}: {value}")

num_species: 1
num_parameters: 11
num_compartments: 1
num_reactions: 1
num_functions: 0
species_list: ['Aplasma']


In [4]:
model_name = "PBPK_BPA_model"
rust_code = converter.convert(model_name)

print("Conversion complete! First 500 characters of generated code:")
print("-" * 50)
print(rust_code[:500])
print("...")

Processing assignment rules...
Found 0 assignment rules
Building ODE system...
Parsing reactions...
Computing Jacobian (sparse)...
Jacobian sparsity: 1/1 non-zero
Optimizing ODE and Jacobian together...
Optimizing expressions with CSE...
DEBUG: Processing 1 CSE replacements for safe divisions...
DEBUG: Total zero-valued symbols found: 0
CSE extracted 1 common subexpressions
Conversion complete! First 500 characters of generated code:
--------------------------------------------------
// Generated WASM-compatible Rust code from SBML model: PBPK_BPA_model
// Uses SymPy CSE for optimized derivatives and Jacobian

use diffsol::{OdeBuilder, OdeSolverMethod, OdeSolverStopReason, Vector};
use wasm_bindgen::prelude::*;
use serde::{Deserialize, Serialize};
use std::collections::HashMap;

type M = diffsol::NalgebraMat<f64>;
type LS = diffsol::NalgebraLU<f64>;

#[derive(Serialize, Deserialize)]
pub struct SimulationResult {
    pub species: std::collections::HashMap<String, Vec<f64>>,

...


In [5]:
with open(f"output/{model_name}.rs", "w") as f:
     f.write(rust_code)
print(f"Saved to {model_name}.rs")

Saved to PBPK_BPA_model.rs
